reference:
> Feature Mode Decomposition: New Decomposition Theory for Rotating Machinery
 Fault Diagnosis

Flow diagram of FMD  

1. load raw signal ```x```, Iput decomposed mode number ```n```, filter length ```L```

2. Initialize FIR filter band with ```K``` filters and start iteration ```i=1```

3. Obtain filtered signal or decomposed mode  $u_k^i$   ```Node 3```

4. Period Estimation 
    + compute autocorrelation $R_k^i$ of decomposed mode $u_k^i$
    + find the local maximum value of $R_k^i$ to get estimated periode $T_k^i$



In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin, lfilter, correlate, find_peaks,hilbert
import pandas as pd

In [3]:
FilterSize = 30 # filter length L
CutNum = 7 #the cut number of the whole frequency band
ModeNum = 2 #the final mode number
MaxIterNum = 20 # max iteration number
fs = 2e4 #sampling frequency of x
signal_x = np.array(pd.read_csv("data/3号机组抽水态健康样本.csv",encoding="GBK").iloc[:,1])



array([6.0712682 , 5.51219438, 3.88398143, ..., 1.76058773, 2.34314462,
       1.46435792])

In [ ]:
class FMD_reconsitution():

    def __init__(self, FilterSize, CutNum,ModeNum,MaxIterNum,fs=None):
        
        self.FilterSize = FilterSize
        self.CutNum= CutNum
        self.ModeNum = ModeNum
        self.MaxIterNum =MaxIterNum
        self.fs = 2e4 if fs ==None else fs

    def initialize_filters(self,L, K):
        
        filters = []
        K = self.CutNum
        eps = np.finfo(float).eps
        freq_bound = np.arange(0, 1, 1 / K)
        L= self.FilterSize
        filters = [
            firwin(FilterSize, [freq_bound[n] + eps, freq_bound[n] + 1 / CutNum - eps],w="hann")
            for n in range(len(freq_bound))
        ]
        
        return filters
    
    def result_initialization(self):
        
        result = pd.DataFrame(
            [[None] * 5 for _ in range(self.CutNum + 1)],
            columns=['IterCount', 'Iterations', 'CorrMatrix', 'ComparedModeNum', 'StopNum']
        )
        
        return result
    
    def update_FIR(self,signal,filters):

        updated_filters = []
        for f in filters:
            period = self.estimate_period(np.convolve(signal, f, mode='same'))
            low_cutoff = max(0, 1 / period - 0.1)
            high_cutoff = min(0.5, 1 / period + 0.1)
            updated_filters.append(firwin(len(f), [low_cutoff, high_cutoff], pass_zero=False, window='hann'))
        
        return updated_filters
   
    def T_estimated(self,signal_x):

        u = np.abs(hilbert(signal_x)) - np.mean(np.abs(hilbert(signal_x)))
        
        return np.argmax(np.correlate(u,u,"full")[int(len(u)//2):])
        # return np.correlate(u,u,"full")[int(len(u)//2):]


    def CK(self,x = None,T = None,M = 2): 

        x = np.array(x).flatten()
        N = len(x)

        x_shift = np.zeros((M + 1,N))
        x_shift[0,:] = x

        for m in range(M):
            if T < N: x_shift[m + 1,T+1:] = x_shift[m,:-T-1]
        
        numerator = np.sum(np.prod(x_shift, axis=0)**2)
        denominator = np.sum(x**2)**(M + 1)
        ck = numerator / denominator
        return ck

    def estimate_period(self,x):
        
        signal = x
        correlation = correlate(signal, signal, mode='full')
        correlation = correlation[len(correlation) // 2:]
        peaks, _ = find_peaks(correlation)
        if len(peaks) > 1:
            period = peaks[1]
        else:
            period = len(signal)
        
        return period
    
    def decompose_modes(x, filters):
        modes = []
        for filt in filters:
            # 对信号x应用每个滤波器
            mode = lfilter(filt, [1.0], x)
            modes.append(mode)
        return modes

    def processing(self,signal_x):
        x = signal_x
        if self.FilterSize % 2 == 0:self.FilterSize += 1  
        temp_filters = self.initialize_filters()
        result = self.result_initialization()

        Final_Mode = None
        return Final_Mode

In [11]:
NA=FMD_reconsitution(
    FilterSize=FilterSize,
    CutNum=CutNum,
    ModeNum=ModeNum,
    fs=None,
    MaxIterNum =MaxIterNum
).T_estimated(np.abs(hilbert(signal_x)) - np.mean(np.abs(hilbert(signal_x))))

In [17]:
all(x<y for x,y in zip([i for i in range(5)],[i for i in range(1,6)]))

True

In [ ]:
all(x<y for x,y in zip(NA,NA[1:]))

False

In [20]:
FMD_reconsitution(
    FilterSize=FilterSize,
    CutNum=CutNum,
    ModeNum=ModeNum,
    fs=None,
    MaxIterNum =MaxIterNum
).initialize_filters(FilterSize,CutNum)[0]

array([-0.00000000e+00,  1.94937430e-04,  8.32285435e-04, -1.99585495e-03,
       -3.78011247e-03,  6.29867633e-03,  9.69988804e-03, -1.41958342e-02,
       -2.01189868e-02,  2.80411933e-02,  3.90514578e-02, -5.55093872e-02,
       -8.36052042e-02,  1.46155507e-01,  4.48931435e-01,  4.48931435e-01,
        1.46155507e-01, -8.36052042e-02, -5.55093872e-02,  3.90514578e-02,
        2.80411933e-02, -2.01189868e-02, -1.41958342e-02,  9.69988804e-03,
        6.29867633e-03, -3.78011247e-03, -1.99585495e-03,  8.32285435e-04,
        1.94937430e-04, -0.00000000e+00])

In [21]:
FMD_reconsitution(
    FilterSize=FilterSize,
    CutNum=CutNum,
    ModeNum=ModeNum,
    fs=None,
    MaxIterNum =MaxIterNum
).result_initialization()

,IterCount,Iterations,CorrMatrix,ComparedModeNum,StopNum
0,None,None,None,None,None
1,None,None,None,None,None
2,None,None,None,None,None
3,None,None,None,None,None
4,None,None,None,None,None
5,None,None,None,None,None
6,None,None,None,None,None
7,None,None,None,None,None


In [ ]:
#汉宁窗口初始化FIR滤波器组
def initialize_filters(L, K):
    filters = []
    for k in range(1, K+1):
        cutoff = 0.5 / k
        filter = firwin(L, cutoff, window='hann')
        filters.append(filter)
    return filters
#自相关普
def estimate_period(signal):
    correlation = correlate(signal, signal, mode='full')
    correlation = correlation[len(correlation) // 2:]
    peaks, _ = find_peaks(correlation)
    if len(peaks) > 1:
        period = peaks[1]
    else:
        period = len(signal)
    return period

 #FMD函数
def fmd(signal, n, L=30, max_iters=10):
    K = min(10, max(5, n))
    filters = initialize_filters(L, K)
    modes = []
    signal = signal.values.flatten() if isinstance(signal, pd.DataFrame) else signal.flatten()
 
    for i in range(max_iters):
        for filter in filters:
            filtered_signal = lfilter(filter, 1.0, signal)
            period = estimate_period(filtered_signal)
            modes.append(filtered_signal)
 
        if len(modes) >= n:
            break
 
    return modes[:n]